# Práctico 5: Generación de Variables Aleatorias Continuas

## Algoritmos

In [1]:
import random as rnd
import numpy as np

### Método de la Transformada Inversa

In [2]:
def inverse_transform_method(G: callable) -> float:
    """
    Inverse Transform Method for generating random numbers from a given distribution.

    Parameters:
    G: callable
        The inverse of the cumulative distribution function of the distribution from which we want to generate random numbers.

    Returns:
    float
        A random number from the distribution.
    """
    U = rnd.random()
    return G(U)

### Generación de VAs Poisson

In [3]:
def poisson_distribution(l: float, sz: int = 1) -> list[int]:
    """
    Generates random numbers from a Poisson distribution.

    Parameters:
    l: float
        The rate parameter of the Poisson distribution.
    sz: int
        The number of random numbers to generate.

    Returns:
    list[int]
        A list of random numbers from the Poisson distribution.
    """
    def generate(l: float) -> int:
        x = 0; p = 1 - rnd.random(); lim = np.exp(-l)
        while p >= lim: p *= 1 - rnd.random(); x += 1
        return x
        
    return [generate(l) for _ in range(sz)]

if __name__ == "__main__":
    # Check if the generated random numbers are from the Poisson distribution.
    l = 2
    sz = 10**5
    random_numbers = poisson_distribution(l, sz)
    print(f"Expected mean: {l} and variance: {l}")
    print(f"Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

Expected mean: 2 and variance: 2
Actual mean: 1.99554 and variance: 2.0075001084000004


### Generación de VAs Gamma

In [4]:
def gamma_distribution(a: int, b: float, sz: int = 1) -> list[float]:
    """
    Generates random numbers from a gamma distribution with shape parameter ('a') as integer.

    Parameters:
    a: int
        The shape parameter of the gamma distribution.
    b: float
        The rate parameter of the gamma distribution.
    sz: int
        The number of random numbers to generate.

    Returns:
    list[float]
        A list of random numbers from the gamma distribution.
    """
    def generate_gamma(a: int, b: float) -> float:
        u = 1
        for _ in range(a): u *= 1 - rnd.random()
        assert u > 0, "Invalid value generated for u"
        return -np.log(u) * b

    return [generate_gamma(a, b) for _ in range(sz)]

if __name__ == "__main__":
    # Check if the generated random numbers are from the gamma distribution.
    a = 2; b = 2
    sz = 10**5
    random_numbers = gamma_distribution(a, b, sz)
    print(f"Expected mean: {a * b} and variance: {a * b ** 2}")
    print(f"Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

Expected mean: 4 and variance: 8
Actual mean: 3.9885881343354046 and variance: 8.002888487682933


### Generación de VAs Exponenciales

In [5]:
def exponential_distribution(l: float, sz: int = 1, gamma_method: bool = True) -> list[float]:
    """
    Generates random numbers from an exponential distribution.

    Parameters:
    l: float
        The rate parameter of the exponential distribution.
    sz: int
        The number of random numbers to generate.
    gamma_method: bool
        If True, uses the gamma distribution method to generate random numbers. Otherwise, uses the inverse transform method.
        (The gamma method is more efficient for generating multiple random numbers but if the SZ is big, -log(u) can be very close to 0 and cause numerical issues.)

    Returns:
    list[float]
        A list of random numbers from the exponential distribution.
    """
    if sz == 1 or not gamma_method:
        return [inverse_transform_method(lambda u: -np.log(1 - u) / l) for _ in range(sz)]

    # Method using a Gamma distribution.
    t = gamma_distribution(sz, 1/l)[0]
    u = sorted([rnd.random() for _ in range(sz - 1)] + [0, 1])
    return [(u[i] - u[i-1]) * t for i in range(1, len(u))]


if __name__ == "__main__":
    # Check if the generated random numbers are from the exponential distribution.
    l = 2
    print(f"Expected mean: {1 / l} and variance: {1 / l ** 2}")
    
    print(f"With Gamma method:")
    sz = 600
    random_numbers = exponential_distribution(l, sz)
    print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

    print(f"With Inverse Transform method:")
    sz = 10**4
    random_numbers = exponential_distribution(l, sz, gamma_method=False)
    print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

Expected mean: 0.5 and variance: 0.25
With Gamma method:
    Actual mean: 0.5284678590183732 and variance: 0.28921699778438886
With Inverse Transform method:
    Actual mean: 0.49447696014382264 and variance: 0.24740045176945413


### Método de Aceptación y Rechazo

In [6]:
def rejection_method(Y: callable, p: callable, q: callable, c: float, sz: int = 1) -> list[float]:
    """
    Estimate the expected value of a function using the rejection method.

    Parameters:
    Y: callable - Function to simulate the random variable.
    p: callable - Probability function of the random variable of X.
    q: callable - Probability function of the random variable of Y.
    c: float - Constant such that p(x) <= c * q(x) for all x.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(Y: callable, p: callable, q: callable, c: float) -> float:
        while True:
            x = Y()
            U = rnd.random()
            if U < p(x) / (c * q(x)): return x
    
    return [generate(Y, p, q, c) for _ in range(sz)]

def transform_rejection_method(H: callable, Hd: callable, p: callable, c: float, sz: int = 1) -> list[float]:
    """
    Estimate the expected value of a function using the rejection method.

    Parameters:
    H: callable - Inverse of the cumulative distribution function of the random variable of Y.
    Hd: callable - Derivative of H.
    p: callable - Probability function of the random variable of X.
    c: float - Constant such that p(x) <= c * q(x) for all x.
    sz: int - Number of random numbers to generate.

    Returns:
    list[int] - List of random numbers.
    """
    def generate(H: callable, Hd: callable, p: callable, c: float) -> float:
        while True:
            U = rnd.random(); V = rnd.random()
            HU = H(U); HdU = Hd(U)
            if V < p(HU) * HdU / c: return HU
    
    return [generate(H, Hd, p, c) for _ in range(sz)]

if __name__ == "__main__":
    # Check to generate Gamma(3/2, 1) with Y as exponential(2/3)
    a = 3/2; b = 1
    Y = lambda: exponential_distribution(2/3)[0]
    p = lambda x : 2/np.pi * x**(1/2) * np.exp(-x)
    q = lambda x : 2/3 * np.exp(-2/3 * x)

    sz = 10**5
    random_numbers = rejection_method(Y, p, q, 1.35, sz)
    print(f"Using Rejection Method:")
    print(f"    Expected mean: {a * b} and variance: {a * b ** 2}")
    print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

    # Check to generate Gamma(3/2, 1) with Y as exponential(2/3)
    H = lambda x: -3/2 * np.log(1 - x)
    Hd = lambda x: 3/2 / (1 - x)
    c = 3 * np.sqrt(3/(2*np.pi*np.e))
    random_numbers = transform_rejection_method(H, Hd, p, c, sz)
    print(f"Using Transform Rejection Method:")
    print(f"    Expected mean: {a * b} and variance: {a * b ** 2}")
    print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

Using Rejection Method:
    Expected mean: 1.5 and variance: 1.5
    Actual mean: 1.5069984128926786 and variance: 1.5081130955669826
Using Transform Rejection Method:
    Expected mean: 1.5 and variance: 1.5
    Actual mean: 1.4987081916679301 and variance: 1.506125699108363


### Generación de VAs Normales

In [7]:
def normal_distribution(m: float, s: float, sz: int = 1, method: str = "rejection") -> list[float]:
    """
    Generates random numbers from a normal distribution.

    Parameters:
    m: float
        The mean of the normal distribution.
    s: float
        The standard deviation of the normal distribution.
    sz: int
        The number of random numbers to generate.
    method: str
        The method to use for generating random numbers. Can be "rejection", "polar", "box-muller" or "uniform_ratios".
        Default is "rejection".

    Returns:
    list[float]
        A list of random numbers from the normal distribution.
    """
    def rejection_method() -> float:
        while True:
            u, y = -np.log(rnd.random()), -np.log(rnd.random())
            if y >= (u - 1) ** 2 / 2: return (u if rnd.random() < 0.5 else -u)
    
    def polar_method() -> list[float]:
        r2 = -2 * np.log(1 - rnd.random())
        theta = 2 * np.pi * rnd.random()
        x, y = np.sqrt(r2) * np.cos(theta), np.sqrt(r2) * np.sin(theta)
        return [x, y]
    
    def box_muller_method() -> list[float]:
        while True:
            v1, v2 = 2 * rnd.random() - 1, 2 * rnd.random() - 1
            if v1 ** 2 + v2 ** 2 <= 1:
                s = v1 ** 2 + v2 ** 2
                x, y = v1 * np.sqrt(-2 * np.log(s) / s), v2 * np.sqrt(-2 * np.log(s) / s)
                return [x, y]
        
    def uniform_ratios_method() -> float:
        c = 4 * np.exp(-0.5) / np.sqrt(2.0)
        while True:
            u, y = rnd.random(), 1 - rnd.random()
            z = c * (u - 0.5) / y
            if z ** 2 / 4 <= -np.log(y): return z
    
    match method:
        case "polar":
            r = []
            while len(r) < sz: r += [m + x * s for x in polar_method()]
            return r[:sz]
        case "box-muller":
            r = []
            while len(r) < sz: r += [m + x * s for x in box_muller_method()]
            return r[:sz]
        case "uniform_ratios":
            return [m + uniform_ratios_method() * s for _ in range(sz)]
        case "rejection":
            return [m + rejection_method() * s for _ in range(sz)]
        case _:
            raise ValueError("Invalid method. Please use 'rejection', 'polar', 'box-muller' or 'uniform_ratios'.")
    
if __name__ == "__main__":
    # Check if the generated random numbers are from the normal distribution.
    m_list = [-2, 2, 5, 10]
    s_list = [1, 1, 2, 3]
    sz = 10**5

    for i in range(len(m_list)):
        m, s = m_list[i], s_list[i]

        # Case 1: Using polar method.
        random_numbers = normal_distribution(m, s, sz, "polar")
        print(f"Using Polar method:")
        print(f"    Expected mean: {m} and variance: {s ** 2}")
        print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

        # Case 2: Using Box-Muller method.
        random_numbers = normal_distribution(m, s, sz, "box-muller")
        print(f"Using Box-Muller method:")
        print(f"    Expected mean: {m} and variance: {s ** 2}")
        print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

        # Case 3: Using Uniform Ratios method.
        random_numbers = normal_distribution(m, s, sz, "uniform_ratios")
        print(f"Using Uniform Ratios method:")
        print(f"    Expected mean: {m} and variance: {s ** 2}")
        print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

        # Case 4: Using Rejection method.
        random_numbers = normal_distribution(m, s, sz, "rejection")
        print(f"Using Rejection method:")
        print(f"    Expected mean: {m} and variance: {s ** 2}")
        print(f"    Actual mean: {np.mean(random_numbers)} and variance: {np.var(random_numbers)}")

        print("\n")



Using Polar method:
    Expected mean: -2 and variance: 1
    Actual mean: -1.9990025552755324 and variance: 1.0038155108414992
Using Box-Muller method:
    Expected mean: -2 and variance: 1
    Actual mean: -2.001054381654991 and variance: 0.9969341551131323
Using Uniform Ratios method:
    Expected mean: -2 and variance: 1
    Actual mean: -1.9994363778520359 and variance: 0.9975360949543333
Using Rejection method:
    Expected mean: -2 and variance: 1
    Actual mean: -1.994271266201237 and variance: 0.9954956774485472


Using Polar method:
    Expected mean: 2 and variance: 1
    Actual mean: 2.0022988695821886 and variance: 0.9979545439907657
Using Box-Muller method:
    Expected mean: 2 and variance: 1
    Actual mean: 1.9981063507106704 and variance: 1.005309533523191
Using Uniform Ratios method:
    Expected mean: 2 and variance: 1
    Actual mean: 2.0039997336241675 and variance: 0.9932063108607804
Using Rejection method:
    Expected mean: 2 and variance: 1
    Actual mean: 1

### Generación de Proceso de Poisson

#### Homogéneo

In [8]:
def poisson_process_events(l: float, t: float) -> list[float]:
    """
    Generates the events of a Poisson process until a given time 't'.

    Parameters:
    l: float
        The rate parameter of the Poisson process.
    t: float
        The time until which the events are generated.

    Returns:
    list[float]
        A list of times at which the events occur.
    """
    x = 0; events = []
    while True:
        x += exponential_distribution(l)[0]
        if x > t: break
        events.append(x)
    return events

if __name__ == "__main__":
    # Check if the generated events are from a Poisson process.
    l = 2
    t = 10
    sz = 10**4
    cnt_events = [len(poisson_process_events(l, t)) for _ in range(sz)]
    print(f"Expected mean: {l * t}")
    print(f"Actual mean: {np.mean(cnt_events)}")

Expected mean: 20
Actual mean: 20.0805


#### No homogéneo (*adelgazamiento*)

In [9]:
def poisson_no_homogeneous_process_events(fl: callable, l: float, t: float) -> list[float]:
    """
    Generates the events of a non-homogeneous Poisson process until a given time 't'.

    Parameters:
    fl: callable
        The function that returns the rate parameter at time 't'.
    l: float
        The upper bound of the rate parameter.
    t: float
        The time until which the events are generated.

    Returns:
    list[float]
        A list of times at which the events occur.
    """
    x = 0; events = []
    while True:
        x += exponential_distribution(l)[0]
        if x > t: break
        if rnd.random() < fl(x) / l: events.append(x)
    return events

def poisson_no_homogeneous_process_events_optimized(fl: callable, l: list[float], t: list[float]) -> list[float]:
    """
    Generates the events of a non-homogeneous Poisson process with time intervals until a given time 't[-1]'.

    Parameters:
    fl: callable
        The function that returns the rate parameter at time 't'.
    l: list[float]
        The upper bound of the rate parameter.
    t: list[float]
        The time until which the events are generated (intervals).
    
    Returns:
    list[float]
        A list of times at which the events occur.
    """
    x = 0; events = []; i = 0
    while True:
        x += exponential_distribution(l[i])[0]
        while i != len(t) - 1 and x > t[i]: x = t[i] + (x - t[i]) * l[i] / l[i+1]; i += 1
        if x > t[-1]: break
        if rnd.random() < fl(x) / l[i]: events.append(x)
    return events

if __name__ == "__main__":
    # Check if the generated events are from a non-homogeneous Poisson process.
    fl = lambda x: 2 + np.sin(x)
    l = 3
    t = 10
    sz = 10**4
    cnt_events = [len(poisson_no_homogeneous_process_events(fl, l, t)) for _ in range(sz)]
    print(f"Using the basic method:")
    print(f"    Expected mean: {t * np.mean([fl(x) for x in np.linspace(0, t, 1000)])}")
    print(f"    Actual mean: {np.mean(cnt_events)}")

    l = [2 + np.sin(x) for x in np.linspace(0, 10, 1000)]
    t = np.linspace(0, 10, 1000)
    cnt_events = [len(poisson_no_homogeneous_process_events_optimized(fl, l, t)) for _ in range(sz)]
    print(f"Using the optimized method:")
    print(f"    Expected mean: {t[-1] * np.mean([fl(x) for x in np.linspace(0, t[-1], 1000)])}")
    print(f"    Actual mean: {np.mean(cnt_events)}")

Using the basic method:
    Expected mean: 21.834497011030297
    Actual mean: 21.8399
Using the optimized method:
    Expected mean: 21.834497011030297
    Actual mean: 21.8679
